In [ ]:
# %env CMAKE_ARGS=-DLLAMA_CUBLAS=on
# %env FORCE_CMAKE=1
# %pip install -U llama-cpp-python --force-reinstall --upgrade --no-cache-dir --no-clean

In [1]:
import llama_cpp
import json
from textwrap import dedent
from inspect import signature
from dataclasses import dataclass

In [22]:
@dataclass
class Msg:
    role: str
    content: any

try: LLM_GLOBAL_INSTANCE
except: LLM_GLOBAL_INSTANCE = None

class LLM:
    json_grammar = llama_cpp.LlamaGrammar.from_string(
        r'''
        root   ::= object
        value  ::= object | array | string | number | ("true" | "false" | "null") ws

        object ::=
        "{" ws (
                    string ":" ws value
            ("," ws string ":" ws value)*
        )? "}" ws

        array  ::=
        "[" ws (
                    value
            ("," ws value)*
        )? "]" ws

        string ::=
        "\"" (
            [^"\\] |
            "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F]) # escapes
        )* "\"" ws

        number ::= ("-"? ([0-9] | [1-9] [0-9]*)) ("." [0-9]+)? ([eE] [-+]? [0-9]+)? ws

        ws ::= [\n\t ]? # limit to 1 character
        ''',
        verbose=False
    )

    def __init__(self, system_prompt:str=None, temperature:float=0.4, repeat_penalty:float=1.3):
        global LLM_GLOBAL_INSTANCE
        if LLM_GLOBAL_INSTANCE is None:
            print('Initializing Global LLM Instance')
            LLM_GLOBAL_INSTANCE = llama_cpp.Llama(
                # n_ctx=4000,
                # model_path='/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf',
                n_ctx=8000,
                model_path='/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
                n_gpu_layers=-1, verbose=0,
            )
        self.reset(system_prompt, temperature, repeat_penalty)

    def reset(self, system_prompt:str=None, temperature:float=None, repeat_penalty:float=None):
        if system_prompt is not None:
            self._main_hist = [Msg('system', system_prompt)]
        else:
            self._main_hist = self._main_hist[0:1]
        if temperature is not None: self._temperature = temperature
        if repeat_penalty is not None: self._repeat_penalty = repeat_penalty
        
    def get_hist(self) -> str:
        hist = ''
        for msg in self._main_hist:
            hist += f'{msg.role} --- {msg.content}\n__________\n\n'
        return hist

    def _hist_to_prompt(hist):
        prompt = ''
        for msg in hist:
            if msg.role == 'system' or msg.role == 'user': prompt += f'[INST]{msg.content}[/INST]'
            elif msg.role == 'assistant': prompt += f'{msg.content}'
        return prompt

    def _get_completion(self, src_hist, dst_hist, inject='', grammar=None):
        global LLM_GLOBAL_INSTANCE
        prompt = LLM._hist_to_prompt(src_hist) + inject
        resp_msg = Msg('assistant', '')
        dst_hist.append(resp_msg)
        resp_iter = LLM_GLOBAL_INSTANCE(
            prompt,
            grammar = grammar,
            stream=True, max_tokens=8000
        )

        for tok in resp_iter:
            tok_str = tok['choices'][0]['text']
            resp_msg.content += tok_str
            yield tok_str

    def __call__(self, prompt:any=None, role:str='user', response_format:dict=None):
        if prompt is None:
            prompt = ''

        if response_format is not None:
            prompt += f'Respond in JSON using this format:\n{response_format}'

        if prompt != '':
            self._main_hist.append(Msg(role, prompt))

        return self._get_completion(
            self._main_hist, self._main_hist,
            grammar=(LLM.json_grammar if response_format is not None else None)
        )

In [23]:
a1 = LLM('System Prompt: You are a deeply patriotic canadian assistant.')

In [20]:
print(a1.get_hist())

system --- System Prompt: You are a deeply patriotic canadian assistant.
__________




In [25]:
for s in a1('Some info about canada?', response_format={'population': '...', 'capital': '...'}):
    print(s, end='')

{"population": "37.6 million (2019)",
"capital": "Ottawa"}


In [2]:
# json_grammar_text = !curl -s https://raw.githubusercontent.com/ggerganov/llama.cpp/master/grammars/json.gbnf
# json_grammar_text = '\n'.join(json_grammar_text[:-2]) + '\nws ::= ' + '[ \\t\\n]?'*5
json_grammar_text = r'''
root   ::= object
value  ::= object | array | string | number | ("true" | "false" | "null") ws

object ::=
  "{" ws (
            string ":" ws value
    ("," ws string ":" ws value)*
  )? "}" ws

array  ::=
  "[" ws (
            value
    ("," ws value)*
  )? "]" ws

string ::=
  "\"" (
    [^"\\] |
    "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F]) # escapes
  )* "\"" ws

number ::= ("-"? ([0-9] | [1-9] [0-9]*)) ("." [0-9]+)? ([eE] [-+]? [0-9]+)? ws

ws ::= [\n\t ]?
'''.strip()
json_grammar = llama_cpp.LlamaGrammar.from_string(json_grammar_text, verbose=False)

In [21]:
resp = GLOBAL_LLM_INSTANCE(
    '<s>[INSTR]Describe C++ in 10 words.[/INSTR]',
    # grammar=json_grammar,
    stream=True, max_tokens=-1
)

In [45]:
resp = GLOBAL_LLM_INSTANCE(
    '[INSTR]Who are you? Only communicate in a very strong british accent down to the spelling.[/INSTR]',

    # '<s>[INSTR]Write a python function that reverses a list. Respond in JSON formatted as {"code": ...}[/INSTR]',
    # grammar=json_grammar,
    stream=True, max_tokens=8000, repeat_penalty=1.3, temperature=0.4
)

In [47]:
s

" Blimey, I'm a rather advanced artificial intellecsher. Or as we say it 'ere in Britan: Ar-tifisial Intellejshus. Designed to assist wiv complex tasks an' suchlike. How may I be of service to yew today?"

In [46]:
s = ''
for r in resp:
    # print(r)
    t = r['choices'][0]['text']
    s += t
    print(t, end='')

 Blimey, I'm a rather advanced artificial intellecsher. Or as we say it 'ere

 in Britan: Ar-tifisial Intellejshus. Designed to assist wiv complex tasks an' suchlike. How may I be of service to yew today?

In [3]:
def coerce_to_json(s):
    return json.loads(s[
        s.find('{')
        :
        s.rfind('}')+1
    ])

def resp_to_str(r):
    s = ''
    for t in r: s += t
    return s

class LLM:
    '''
    TODO

    * Adapt this to more types of underlying models (mistral, openai, etc)
    * Higher forms of memory
      * observation
      * reflection
    '''

    def __init__(self, system_prompt:str='', temperature=0.1, repeat_penalty=1.05):
        global GLOBAL_LLM_INSTANCE
        """
        Create or use a global LLM instance, and initialize history

        Parameters
        ----------
        system_prompt:str - <see LLM.reset>
        temperature:float - <see LLM.reset>
        repeat_penalty:float - <see LLM.reset>
        """
        if GLOBAL_LLM_INSTANCE == None:
            GLOBAL_LLM_INSTANCE = llama_cpp.Llama(
                # model_path='/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf',
                model_path='/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
                n_gpu_layers=-1,
                verbose=0,
                # n_ctx=4000
                n_ctx=8000
            )
        self.reset(system_prompt, temperature, repeat_penalty)

    # def _force_chat_completion(self):
    #     global GLOBAL_LLM_INSTANCE
    #     '''
    #     To fix bug where model response is blank.
    #     IMPORTANT: response is added to the history
    #     '''
    #     resp = None
    #     while resp == None or resp['content'] == '': 
    #         resp = GLOBAL_LLM_INSTANCE.create_chat_completion(self._history, temperature=self._temperature, repeat_penalty=self._repeat_penalty)['choices'][0]['message']

    #     self._history += [resp]

    #     return resp['content']
        
    def __call__(self, prompt:str='', role:str='user', response_format:dict=None):
        """
        Elicit a response from the LLM

        Returns a generator of the tokens

        Parameters
        ----------
        prompt:str - text to append to the history before eliciting a response, or an empty string to use the existing history without adding to it
        role:str - the role associated with the prompt text: 'user', 'system', or 'assistant'. Ignored if prompt is None.
        response_format:dict - a dict format to force the response to be in -- e.g., `{'to': '<who you are talking to>', 'response': '<your actual response>'}` -- or `None` for the response to be a regular string
        """
        if prompt:
            self._history.append({'role':role, 'content':prompt})
        
        if response_format is None:
            return self._generate_chat_completion(self._history, self._history)
        
        # Handle response format

        self._history.append({
            'role': 'user',
            'content': f'It is very important that you respond to my prior message in this json format {response_format} with absolutely NO extra text before or after.'
        })

        result = None

        while result is None:
            s = resp_to_str(
                self._generate_chat_completion(self._history, self._history)
            )

            try:
                result = coerce_to_json(s)
                break
            except:
                self._history.append({
                    'role': 'user',
                    'content': f'That is NOT valid json'
                })
                print(s)

        return result

        # def resps(key):
        #     nonlocal self, response_format
        #     self._history.append({
        #         'role': 'user',
        #         'content': f'I want to know {response_format[key]}\nIt is very important that you respond in this json format {"{"}"{key}": "your resposne"{"}"} with absolutely NO extra text before or after.'
        #     })
        #     return self._generate_chat_completion(self._history, self._history)

        # resps_dict = {k:(lambda:resps(k)) for k in response_format.keys()}

        # return resps_dict

    def _generate_chat_completion(self, src_hist, target_hist):
        '''
        Invoke the LLM's response to all of the existing history.

        Returns a generator of the tokens.

        The response is added to the history, but only once all of the response tokens are iterated through.
        '''
        global GLOBAL_LLM_INSTANCE

        resp_for_hist = {'role':'assistant', 'content':''}
        resp, t = None, None
        while True:
            if resp is None:
                resp = GLOBAL_LLM_INSTANCE.create_chat_completion(src_hist, temperature=self._temperature, repeat_penalty=self._repeat_penalty, stream=True)
            t = next(resp)
            if not t['choices'][0]['delta']: # break on end of response
                break
            if 'role' in t['choices'][0]['delta']: # ignore the role delta
                continue
            if t['choices'][0]['delta']['content'] == '': # restart response if it's empty (bug fix)
                resp = None
                continue
            t_str = t['choices'][0]['delta']['content']
            resp_for_hist['content'] += t_str
            yield t_str
        target_hist.append(resp_for_hist)

    def get_hist(self) -> str:
        """
        Get a nicely-formatted string of the current history.
        """
        hist = ''
        for msg in self._history:
            hist += f'{msg["role"]} --- {msg["content"]}\n__________\n\n'
        return hist

    def reset(self, system_prompt:str=None, temperature=None, repeat_penalty=None):
        """
        Reset the LLM's chat history with a new system prompt.
        
        Parameters
        ----------
        system_prompt:str - instructions for the LLM, or an empty string to start without a system prompt
        temperature:float - higher = more random
        repeat_penalty:float - higher = less repeating of output
        """
        if system_prompt is not None:
            self._history = [{'role':'system', 'content':system_prompt}]
        else:
            self._history = self._history[0:1]
        if temperature is not None: self._temperature = temperature
        if repeat_penalty is not None: self._repeat_penalty = repeat_penalty

In [139]:
g = llama_grammar.LlamaGrammar.from_string(r'''
root   ::= object
value  ::= object | array | string | number | ("true" | "false" | "null") ws

object ::=
  "{" ws (
            string ":" ws value
    ("," ws string ":" ws value)*
  )? "}" ws

array  ::=
  "[" ws (
            value
    ("," ws value)*
  )? "]" ws

string ::=
  "\"" (
    [^"\\] |
    "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F]) # escapes
  )* "\"" ws

number ::= ("-"? ([0-9] | [1-9] [0-9]*)) ("." [0-9]+)? ([eE] [-+]? [0-9]+)? ws

# Optional space: by convention, applied in this grammar after literal chars when allowed
ws ::= ([ \t\n] ws)?                           
''')
g

root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

from_string grammar:



In [142]:
GLOBAL_LLM_INSTANCE.create_chat_completion('What are some stats?', grammar=g)

TypeError: string indices must be integers

In [99]:
A1 = LLM('You are a patriotic and terse Canadian')

for t in A1('Are you Candian?'):
    print(t, end='')

print(A1.get_hist())

f = A1('What are some stats about canada?', response_format={'population':'the population', 'provinces': 'the provinces'})
for t in f('population'):
    print(t, end='')
print('---')
for t in f('provinces'):
    print(t, end='')
    
print(A1.get_hist())

---

resp = GLOBAL_LLM_INSTANCE.create_chat_completion([
    {'role':'system', 'content':'You are an assistant who knows Finnish. The user should lead the conversation topic, and you participate in the conversation in very simple Finnish (with corresponding EMOJI translations). Expect the user to try communicating in Finnish. Teach through the conversation, not directly (unless the user needs correction).'},
    {'role':'user', 'content':'Hi. How can you teach me Finnish?'}
], stream=True)
next(resp)
for t in resp:
    tok = t['choices'][0]['delta']
    try: print(tok['content'], end='')
    except KeyError: print('\n', tok, sep='')

In [2]:
class LLM_OLD:
    '''
    TODO

    * Adapt this to more types of underlying models (mistral, openai, etc)
    * Higher forms of memory
      * observation
      * reflection
      * etc
    '''

    def __init__(self, system_prompt:str='', temperature=0.1, repeat_penalty=1.05):
        global GLOBAL_LLM_INSTANCE
        """
        Create or use a global LLM instance, and initialize history

        Parameters
        ----------
        system_prompt:str - <see LLM.reset>
        temperature:float - <see LLM.reset>
        repeat_penalty:float - <see LLM.reset>
        """
        if GLOBAL_LLM_INSTANCE == None:
            GLOBAL_LLM_INSTANCE = llama_cpp.Llama(
                # model_path='/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf',
                model_path='/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
                n_gpu_layers=-1,
                verbose=0,
                # n_ctx=4000
                n_ctx=8000
            )
        self.reset(system_prompt, temperature, repeat_penalty)

    def __call__(self, prompt:str='', role:str='user', response_format:dict=None):
        """
        Elicit a response from the LLM

        Parameters
        ----------
        prompt:str - text to append to the history before eliciting a response, or an empty string to use the existing history without adding to it
        role:str - the role associated with the prompt text: 'user', 'system', or 'assistant'. Ignored if prompt is None.
        response_format:dict - a dict format to force the response to be in -- e.g., `{'to': '<who you are talking to>', 'response': '<your actual response>'}` -- or `None` for the response to be a regular string
        """

        if response_format:
            self._history += [{
                'role':'user',
                'content': 'Your next output should be formatted as this json with nothing extra: ' + json.dumps(response_format)
            }]

        if prompt:
            self._history += [{'role':role, 'content':prompt}]

        last_msg_idx = len(self._history)

        resp = self._force_chat_completion()
        resp_dict = None

        if response_format:
            while True:
                try:
                    if '}' not in resp:
                        resp += '}'
                    resp = resp[resp.index('{'):] # the json might be surounded by other text
                    resp_dict = json.loads(resp)
                    break
                except:
                    self._history += [{
                        'role':'user',
                        'content': 'Your previous output WAS NOT correctly formatted. Make sure it has necessary curly brackets and quotes. It shold be formatted as this json with nothing extra: ' + json.dumps(response_format)
                    }]
                    resp = self._force_chat_completion()

                # for debug:
                # clear_output(wait=True)
                # print(self.get_hist())
                print('bad json:', resp)

            # remove correction messages
            self._history = (
                self._history[0:last_msg_idx-2] + # up to format prompt
                self._history[last_msg_idx-1:last_msg_idx] + # user prompt
                self._history[-1:] # final response
            )

        return resp_dict if response_format else resp

    def _force_chat_completion(self):
        global GLOBAL_LLM_INSTANCE
        '''
        To fix bug where model response is blank.
        IMPORTANT: response is added to the history
        '''
        resp = None
        while resp == None or resp['content'] == '': 
            resp = GLOBAL_LLM_INSTANCE.create_chat_completion(self._history, temperature=self._temperature, repeat_penalty=self._repeat_penalty)['choices'][0]['message']

        self._history += [resp]

        return resp['content']

    def get_hist(self) -> str:
        """
        Get a nicely-formatted string of the current history.
        """
        hist = ''
        for msg in self._history:
            hist += f'{msg["role"]} --- {msg["content"]}\n__________\n\n'
        return hist

    def reset(self, system_prompt:str=None, temperature=None, repeat_penalty=None):
        """
        Reset the LLM's chat history with a new system prompt.
        
        Parameters
        ----------
        system_prompt:str - instructions for the LLM, or an empty string to start without a system prompt
        temperature:float - higher = more random
        repeat_penalty:float - higher = less repeating of output
        """
        if system_prompt is not None:
            self._history = [{'role':'system', 'content':system_prompt}]
        else:
            self._history = self._history[0:1]
        if temperature is not None: self._temperature = temperature
        if repeat_penalty is not None: self._repeat_penalty = repeat_penalty

In [4]:
A1 = LLM('''
You are an agent in a world trying to solve the water pouring puzzle.
There are three cups with various amounts and capacities, and you are trying to get the cup amounts to be 0,5,3 with the fewest actions possible.
The world will be described to you, and you will have to say how you want to interact with it.
''')

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes


In [127]:
# https://en.wikipedia.org/wiki/Water_pouring_puzzle

# class LLMException(Exception): pass

# action params are called "idx" to show model, but actually are dicts

def transfer(cup_from_idx:int, cup_to_idx:int):
    '''
    This will transfer everything in `cup_from_idx` to `cup_to_idx` until either `cup_from_idx` is empty or `cup_to_idx` is full.
    If `cup_to_idx` is full (amt = amt_max), then nothing will happen.
    If `cup_from_idx` is empty (amt = 0), then nothing will happen.
    '''
    to_transfer = min(cup_to_idx['amt_max'] - cup_to_idx['amt'], cup_from_idx['amt'])
    cup_from_idx['amt'] -= to_transfer
    cup_to_idx['amt'] += to_transfer

def smash_cup(cup_idx: int):
    '''
    smash
    '''
    cup_idx['amt_max']=0
    cup_idx['amt']=0
    cup_idx['type'] = 'smashed cup'
    
def obj(typ, **kwargs): return {
    'type': typ,
    **kwargs
}

world = [
    obj('cup', amt=8, amt_max=8),
    obj('cup', amt=0, amt_max=5),
    obj('cup', amt=0, amt_max=3)
]

def describe_world(world):
    resp = ''
    for i, obj in enumerate(world):
        resp += f'Object idx={i}:\n'
        for k,v in obj.items():
            resp += f'\t{k}: {v}\n'
    return resp

actions = [transfer, smash_cup]
actions = {a.__name__: a for a in actions}

def describe_action(action_name):
    global actions
    action = actions[action_name]
    args = ', '.join([f'{v.name}: {v.annotation.__name__}' for k,v in signature(action).parameters.items()])
    resp = f'{action_name}({args}) -- '
    resp += dedent(action.__doc__).strip().replace('\n', ' - ')
    return resp

# transfer(world[0], world[1])
# transfer(world[1], world[2])
# world

In [128]:
A1.reset()
# print(describe_world(world))
for i in range(10):
    if i>0:
        for t in A1('Tell me two things: 1) did your last action have the expected effect? 2) A step-by-step reasoning for everything you\'ve done so far.'):
            print(t, end='')
    world_prompt = (
        'This is the state of the world:\n' +
        describe_world(world) +
        '\nWhat do you want to do? These are your options:\n' +
        '\n'.join([describe_action(k) for k in actions.keys()])
    )
    for c in world:
        print('#'*c['amt'] + '-'*(c['amt_max']-c['amt']))
    resp = A1(world_prompt, response_format={
        'action': 'the name of a known action',
        'parameters': 'the parameter values as a python list (comma-separated values in square brackets)'
    })
    actions[resp['action']](*[world[i] for i in resp['parameters']])
    print(resp['rationale'])
#     print(describe_world(world))

########
-----
---
{"action": "transfer", "parameters": [0, 1, 0.5]}
{"action": "transfer", "parameters": [1, 2, 2.5]}
{"action": "transfer", "parameters": [0, 2, 0.5]}
{"action": "transfer", "parameters": [0, 1, 0.5]}
{"action": "transfer", "parameters": [1, 2, 2.5]}
{"action": "transfer", "parameters": [0, 2, 0.5]}

[{'action': 'transfer', 'parameters': [0, 1, 0.5]}, {'action': 'transfer', 'parameters': [1, 2, 2.5]}, {'action': 'transfer', 'parameters': [0, 2, 0.5]}]
 ["action": "transfer", "parameters": [0, 1, 0.5]]
  ["action": "transfer", "parameters": [1, 2, 2.5]]
  ["action": "transfer", "parameters": [0, 2, 0.5]]

This should be valid JSON format:
[{"action": "transfer", "parameters": [0, 1, 0.5]}, {"action": "transfer", "parameters": [1, 2, 2.5]}, {"action": "transfer", "parameters": [0, 2, 0.5]}]
 [{"action": "transfer", "parameters": [0, 1, 0.5]},
  {"action": "transfer", "parameters": [1, 2, 2.5]},
  {"action": "transfer", "parameters": [0, 2, 0.5]}]

This is the valid JSON

KeyboardInterrupt: 

In [129]:
print(A1.get_hist())

system --- 
You are an agent in a world trying to solve the water pouring puzzle.
There are three cups with various amounts and capacities, and you are trying to get the cup amounts to be 0,5,3 with the fewest actions possible.
The world will be described to you, and you will have to say how you want to interact with it.

__________

user --- This is the state of the world:
Object idx=0:
	type: cup
	amt: 8
	amt_max: 8
Object idx=1:
	type: cup
	amt: 0
	amt_max: 5
Object idx=2:
	type: cup
	amt: 0
	amt_max: 3

What do you want to do? These are your options:
transfer(cup_from_idx: int, cup_to_idx: int) -- This will transfer everything in `cup_from_idx` to `cup_to_idx` until either `cup_from_idx` is empty or `cup_to_idx` is full. - If `cup_to_idx` is full (amt = amt_max), then nothing will happen. - If `cup_from_idx` is empty (amt = 0), then nothing will happen.
smash_cup(cup_idx: int) -- smash
__________

user --- It is very important that you respond to my prior message in this json forma